# Βήμα 1. Συλλογή δεδομένων:


In [4]:
import requests
from bs4 import BeautifulSoup
import json

#Συνάρτηση για να ανακτήσουμε συγκεκριμένα άρθρα
def fetch_articles(start_url, num_articles = 10):
    base_url = "https://en.wikipedia.org/"
    visited = set() #Σύνολο url τα οποία έχουμε ήδη επισκεφτεί
    articles = [] #Δεδομένα των άρθρων
    
    #Συνάρτηση για να συλλέξουμε τα δεδομένα της κάθε σελίδας
    def fetch_article_data(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find('h1').text
        paragraphs = soup.find_all('p')
        content = " ".join([p.text for p in paragraphs])
        return {"title": title, "url": url, "content": content}
    
    #Τα url τα οποία θα επισκεφτούμε στην συνέχεια
    to_visit = [start_url]
    while to_visit and len(articles) < num_articles:
        current_url = to_visit.pop(0)
        if current_url in visited:
            continue
        visited.add(current_url)
        
        #Αποφεύγουμε το Main Page της Wikipedia
        if 'Main_Page' in current_url:
            continue
        
        print(f"Fetching: {current_url}")
        article = fetch_article_data(current_url)
        if article:
            articles.append(article)
        
        response = requests.get(current_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        for link in soup.find_all('a', href = True):
            href = link['href']
            if href.startswith('/wiki/') and ':' not in href:
                full_url = base_url + href
                if full_url not in visited:
                    to_visit.append(full_url)
        
    return articles
        

start_url = "https://en.wikipedia.org/wiki/Processor_(computing)"
articles = fetch_articles(start_url, num_articles = 9)

with open("articles.json", "w", encoding="utf-8") as file:
    json.dump(articles, file, ensure_ascii=False, indent=4)
print("The data has been saved to \"articles.json\"")

Fetching: https://en.wikipedia.org/wiki/Processor_(computing)
Fetching: https://en.wikipedia.org//wiki/Processor_(computing)
Fetching: https://en.wikipedia.org//wiki/Processor_(disambiguation)#Computing
Fetching: https://en.wikipedia.org//wiki/Computing
Fetching: https://en.wikipedia.org//wiki/Computer_science
Fetching: https://en.wikipedia.org//wiki/Circuit_(computer_science)
Fetching: https://en.wikipedia.org//wiki/Memory_(computing)
Fetching: https://en.wikipedia.org//wiki/Microprocessor
Fetching: https://en.wikipedia.org//wiki/Metal%E2%80%93oxide%E2%80%93semiconductor
The data has been saved to "articles.json"


# Βήμα 2. Προεπεξεργασία κειμένου (Text Processing):

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Συνάρτηση για επεξεργασία κειμένου
def process_content(text):
    if not isinstance(text, str):  # Έλεγχος ότι η είσοδος είναι string
        return text
    
    # Tokenization
    words = word_tokenize(text)
    processed_words = []
    
    for word in words:
        if word.isdigit():  # Αν είναι αριθμός, το προσθέτουμε απευθείας
            processed_words.append(word)
        else:
            # Αφαίρεση ειδικών χαρακτήρων
            word = word.strip(string.punctuation)
            
            # Lemmatization
            lemmatizer = WordNetLemmatizer()
            word = lemmatizer.lemmatize(word)
            
            # Αφαίρεση stop words
            stop_words = set(stopwords.words('english'))
            if word.lower() not in stop_words:
                processed_words.append(word.lower())
    
    return ' '.join(processed_words)

# Ανάγνωση δεδομένων από αρχείο JSON
try:
    dataframe = pd.read_json("articles.json")  # διαδρομή αρχείου
except Exception as e:
    print(f"Error reading JSON file: {e}")
    exit()

# Επεξεργασία δεδομένων DataFrame
for column in dataframe.columns:
    dataframe[column] = dataframe[column].apply(process_content)

# Μετονομασία στηλών
new_columns = {
    dataframe.columns[0]: 'Title',
    dataframe.columns[1]: 'Url',
    dataframe.columns[2]: 'Content'
}
dataframe = dataframe.rename(columns=new_columns)

# Αποθήκευση επεξεργασμένων δεδομένων σε νέο αρχείο JSON
dataframe.to_json("processed.json", orient='records', indent=4)

print("The data has been saved to \"processed.json\"")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Μαρία\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Μαρία\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Μαρία\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


The data has been saved to "processed.json"


# Βήμα 3. Ευρετήριο (indexing): 

In [3]:
import json
from collections import defaultdict

# Διαβάζουμε τα επεξεργασμένα δεδομένα από το αρχείο processed.json
with open('processed.json', 'r') as json_file:
    documents = json.load(json_file)  # Φορτώνουμε το JSON σε μορφή λίστας/λεξικού

# Αντίστροφο ευρετήριο (inverted index)
inverted_index = defaultdict(list)  # Χρησιμοποιούμε defaultdict για να αποθηκεύσουμε τα IDs των εγγράφων

# Επεξεργασία κάθε έγγραφου
for doc_id, document in enumerate(documents):
    # Σπλιτάρω και αποθηκεύω στο terms όλα τα κομμάτια
    for field, field_value in document.items():
        if isinstance(field_value, str):  # Ελέγχω αν το πεδίο είναι string
            terms = field_value.split()  # Χωρίζω το κείμενο σε όρους
            
            # Δημιουργία του αντεστραμμένου ευρετηρίου (αντί για αριθμό εμφανίσεων, αποθηκεύουμε τα doc_id)
            for term in terms:
                if doc_id not in inverted_index[term]:  # Ελέγχουμε αν το doc_id είναι ήδη στη λίστα
                    inverted_index[term].append(doc_id)

# Αποθήκευση του αντεστραμμένου ευρετηρίου σε αρχείο JSON
with open('inverted_index.json', 'w') as json_file:
    json.dump(inverted_index, json_file, indent=4)  


print("The data has been saved to \"inverted_index.json\"")

The data has been saved to "inverted_index.json"


# Βήμα 4. Μηχανή αναζήτησης (Search Engine):